# Практическая работа

# Задача

Один из способов повысить эффективность взаимодействия банка с клиентами — отправлять предложение о новой услуге не всем клиентам, а только некоторым, которые выбираются по принципу наибольшей склонности к отклику на это предложение.

Задача заключается в том, чтобы предложить алгоритм, который будет выдавать склонность клиента к положительному или отрицательному отклику на предложение банка. Предполагается, что, получив такие оценки для некоторого множества клиентов, банк обратится с предложением только к тем, от кого ожидается положительный отклик.


Для решения этой задачи загрузите файлы из базы в Postgres.
Эта БД хранит информацию о клиентах банка и их персональные данные, такие как пол, количество детей и другие.

Описание таблиц с данными представлено ниже.


**D_work**

Описание статусов относительно работы:
- ID — идентификатор социального статуса клиента относительно работы;
- COMMENT — расшифровка статуса.


**D_pens**

Описание статусов относительно пенсии:
- ID — идентификатор социального статуса;
- COMMENT — расшифровка статуса.


**D_clients**

Описание данных клиентов:
- ID — идентификатор записи;
- AGE	— возраст клиента;
- GENDER — пол клиента (1 — мужчина, 0 — женщина);
- EDUCATION — образование;
- MARITAL_STATUS — семейное положение;
- CHILD_TOTAL	— количество детей клиента;
- DEPENDANTS — количество иждивенцев клиента;
- SOCSTATUS_WORK_FL	— социальный статус клиента относительно работы (1 — работает, 0 — не работает);
- SOCSTATUS_PENS_FL	— социальный статус клиента относительно пенсии (1 — пенсионер, 0 — не пенсионер);
- REG_ADDRESS_PROVINCE — область регистрации клиента;
- FACT_ADDRESS_PROVINCE — область фактического пребывания клиента;
- POSTAL_ADDRESS_PROVINCE — почтовый адрес области;
- FL_PRESENCE_FL — наличие в собственности квартиры (1 — есть, 0 — нет);
- OWN_AUTO — количество автомобилей в собственности.


**D_agreement**

Таблица с зафиксированными откликами клиентов на предложения банка:
- AGREEMENT_RK — уникальный идентификатор объекта в выборке;
- ID_CLIENT — идентификатор клиента;
- TARGET — целевая переменная: отклик на маркетинговую кампанию (1 — отклик был зарегистрирован, 0 — отклика не было).
    
    
**D_job**

Описание информации о работе клиентов:
- GEN_INDUSTRY — отрасль работы клиента;
- GEN_TITLE — должность;
- JOB_DIR — направление деятельности внутри компании;
- WORK_TIME — время работы на текущем месте (в месяцах);
- ID_CLIENT — идентификатор клиента.


**D_salary**

Описание информации о заработной плате клиентов:
- ID_CLIENT — идентификатор клиента;
- FAMILY_INCOME — семейный доход (несколько категорий);
- PERSONAL_INCOME — личный доход клиента (в рублях).


**D_last_credit**

Информация о последнем займе клиента:
- ID_CLIENT — идентификатор клиента;
- CREDIT — сумма последнего кредита клиента (в рублях);
- TERM — срок кредита;
- FST_PAYMENT — первоначальный взнос (в рублях).


**D_loan**

Информация о кредитной истории клиента:
- ID_CLIENT — идентификатор клиента;
- ID_LOAN — идентификатор кредита.

**D_close_loan**

Информация о статусах кредита (ссуд):
- ID_LOAN — идентификатор кредита;
- CLOSED_FL — текущий статус кредита (1 — закрыт, 0 — не закрыт).

Ниже представлен минимальный список колонок, которые должны находиться в итоговом датасете после склейки и агрегации данных. По своему усмотрению вы можете добавить дополнительные к этим колонки.

    - AGREEMENT_RK — уникальный идентификатор объекта в выборке;
    - TARGET — целевая переменная: отклик на маркетинговую кампанию (1 — отклик был зарегистрирован, 0 — отклика не было);
    - AGE — возраст клиента;
    - SOCSTATUS_WORK_FL — социальный статус клиента относительно работы (1 — работает, 0 — не работает);
    - SOCSTATUS_PENS_FL — социальный статус клиента относительно пенсии (1 — пенсионер, 0 — не пенсионер);
    - GENDER — пол клиента (1 — мужчина, 0 — женщина);
    - CHILD_TOTAL — количество детей клиента;
    - DEPENDANTS — количество иждивенцев клиента;
    - PERSONAL_INCOME — личный доход клиента (в рублях);
    - LOAN_NUM_TOTAL — количество ссуд клиента;
    - LOAN_NUM_CLOSED — количество погашенных ссуд клиента.


Будьте внимательны при сборке датасета: это реальные банковские данные, в которых могут наблюдаться дубли, некорректно заполненные значения или значения, противоречащие друг другу. Для получения качественной модели необходимо предварительно очистить датасет от такой информации.

In [1]:
from app import REMAP_DICT

import pandas as pd
import numpy as np
from catboost import CatBoostClassifier, metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score, recall_score, precision_score

pd.set_option("display.max_columns", None)

## Задание 1

Соберите всю информацию о клиентах в одну таблицу, где одна строчка соответствует полной информации об одном клиенте.

In [2]:
clients_df = pd.read_csv("data/D_clients.csv")
target_df = pd.read_csv("data/D_target.csv")
salary_df = pd.read_csv("data/D_salary.csv")
loan_df = pd.read_csv("data/D_loan.csv")
last_credit_df = pd.read_csv("data/D_last_credit.csv")
job_df = pd.read_csv("data/D_job.csv")
close_loan_df = pd.read_csv("data/D_close_loan.csv")

# D_work и D_pens содержат некорректные расшифровки флагов, поэтому не будем использовать их
# pens_df = pd.read_csv("data/D_pens.csv")
# work_df = pd.read_csv("data/D_work.csv")

In [3]:
df = pd.merge(clients_df, pd.merge(loan_df, close_loan_df, on="ID_LOAN"), left_on="ID", right_on="ID_CLIENT")
df = pd.merge(df, job_df, on="ID_CLIENT")
df = pd.merge(df, target_df, on="ID_CLIENT")
df = pd.merge(df, salary_df, on="ID_CLIENT")
df = pd.merge(df, last_credit_df, on="ID_CLIENT")

df.drop(columns=["ID"], inplace=True)
df.rename(columns={"COMMENT_x": "WORKING", "COMMENT_y": "PENSIONER", "ID_x": "ID_CLIENT",
                  "FL_PRESENCE_FL": "HAS_FLAT", "CLOSED_FL": "CLOSED_CREDIT",
                  "SOCSTATUS_WORK_FL": "IS_WORKING", "SOCSTATUS_PENS_FL": "IS_PENSIONER",
                  "FST_PAYMENT": "FIRST_PAYMENT"}, inplace=True)
# Избавимся от дубликатных объектов в выборке, используя ключ AGREEMENT_RK
df.drop_duplicates(subset=["AGREEMENT_RK"], inplace=True)
df

,AGE,GENDER,EDUCATION,MARITAL_STATUS,CHILD_TOTAL,DEPENDANTS,IS_WORKING,IS_PENSIONER,REG_ADDRESS_PROVINCE,FACT_ADDRESS_PROVINCE,POSTAL_ADDRESS_PROVINCE,HAS_FLAT,OWN_AUTO,ID_LOAN,ID_CLIENT,CLOSED_CREDIT,GEN_INDUSTRY,GEN_TITLE,JOB_DIR,WORK_TIME,AGREEMENT_RK,TARGET,FAMILY_INCOME,PERSONAL_INCOME,CREDIT,TERM,FIRST_PAYMENT
0,42,1,Среднее,Не состоял в браке,1,0,1,0,Московская область,Московская область,Московская область,1,0,1753791446,106805103,0,Другие сферы,Работник сферы услуг,Участие в основ. деятельности,3.0,60099204,1,от 20000 до 50000 руб.,25000.0,5588.0,6,1000.0
1,28,1,Среднее специальное,Состою в браке,1,1,1,0,Читинская область,Читинская область,Читинская область,0,0,1753796120,106809308,0,Торговля,Специалист,Участие в основ. деятельности,5.0,62244665,0,от 10000 до 20000 руб.,10000.0,19498.0,12,0.0
2,64,0,Среднее специальное,Состою в браке,2,0,1,1,Иркутская область,Иркутская область,Иркутская область,0,1,1753792244,106805867,1,Другие сферы,Руководитель высшего звена,Участие в основ. деятельности,360.0,61050759,0,от 20000 до 50000 руб.,30000.0,15470.0,3,15000.0
3,54,1,Среднее специальное,Состою в браке,0,0,1,0,Новосибирская область,Новосибирская область,Новосибирская область,1,1,1753795547,106808779,0,Государственная служба,Специалист,Участие в основ. деятельности,3.0,62079659,0,от 20000 до 50000 руб.,25000.0,13960.0,6,2500.0
4,26,0,Среднее специальное,Состою в браке,1,1,1,0,Красноярский край,Красноярский край,Красноярский край,1,0,1753802107,106814289,1,Другие сферы,Специалист,Участие в основ. деятельности,12.0,66583553,0,от 10000 до 20000 руб.,15000.0,11890.0,6,8000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21529,26,1,Среднее,Не состоял в браке,1,1,1,0,Чувашия,Чувашия,Чувашия,0,0,1753803070,106814680,1,Сборочные производства,Специалист,Участие в основ. деятельности,36.0,66751099,0,от 5000 до 10000 руб.,8000.0,11750.0,4,1900.0
21531,26,0,Среднее специальное,Состою в браке,0,0,1,0,Карелия,Карелия,Карелия,0,0,1753799545,106812464,0,Торговля,Специалист,Участие в основ. деятельности,24.0,64562377,0,от 20000 до 50000 руб.,12000.0,12350.0,6,1380.0
21532,30,1,Среднее специальное,Не состоял в браке,0,0,1,0,Белгородская область,Белгородская область,Белгородская область,0,0,1753796042,106809255,0,Торговля,Специалист,Участие в основ. деятельности,36.0,62236542,0,от 5000 до 10000 руб.,9000.0,4915.0,10,2000.0
21533,25,0,Среднее специальное,Состою в браке,0,0,1,0,Кабардино-Балкария,Кабардино-Балкария,Кабардино-Балкария,0,0,1753802864,106814593,1,Транспорт,Специалист,Участие в основ. деятельности,36.0,66739926,0,от 10000 до 20000 руб.,12000.0,5860.0,3,2000.0


In [4]:
df.describe(percentiles=[0.01, 0.05, 0.1, 0.25, 0.5, 0.75, 0.90, 0.95, 0.99])

,AGE,GENDER,CHILD_TOTAL,DEPENDANTS,IS_WORKING,IS_PENSIONER,HAS_FLAT,OWN_AUTO,ID_LOAN,ID_CLIENT,CLOSED_CREDIT,WORK_TIME,AGREEMENT_RK,TARGET,PERSONAL_INCOME,CREDIT,TERM,FIRST_PAYMENT
count,15223.000000,15223.000000,15223.000000,15223.000000,15223.000000,15223.000000,15223.000000,15223.000000,1.522300e+04,1.522300e+04,15223.000000,1.385500e+04,1.522300e+04,15223.000000,15223.000000,15223.000000,15223.000000,15223.000000
mean,40.406096,0.654536,1.099389,0.645208,0.909610,0.134468,0.309794,0.116337,1.753800e+09,1.068120e+08,0.438613,2.922120e+02,6.540183e+07,0.119030,13853.836323,14667.959345,8.101031,3398.562655
std,11.601068,0.475535,0.995411,0.812252,0.286748,0.341165,0.462424,0.320844,6.087398e+03,4.394646e+03,0.496234,2.436483e+04,4.568181e+06,0.323835,9015.467617,12147.873496,4.094090,5158.108934
min,21.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.753791e+09,1.068044e+08,0.000000,1.000000e+00,5.991015e+07,0.000000,24.000000,2000.000000,3.000000,0.000000
1%,22.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.753791e+09,1.068045e+08,0.000000,3.000000e+00,5.992792e+07,0.000000,4300.000000,2498.220000,3.000000,0.000000
5%,24.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.753791e+09,1.068051e+08,0.000000,6.000000e+00,6.010312e+07,0.000000,5000.000000,3500.000000,3.000000,0.000000
10%,25.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.753792e+09,1.068059e+08,0.000000,1.000000e+01,6.105632e+07,0.000000,6000.000000,4326.420000,3.000000,400.000000
25%,30.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.753795e+09,1.068082e+08,0.000000,2.400000e+01,6.192092e+07,0.000000,8000.000000,6500.000000,6.000000,1000.000000
50%,39.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.753799e+09,1.068120e+08,0.000000,4.800000e+01,6.437100e+07,0.000000,12000.000000,11550.000000,6.000000,2000.000000
75%,50.000000,1.000000,2.000000,1.000000,1.000000,0.000000,1.000000,0.000000,1.753805e+09,1.068158e+08,1.000000,1.100000e+02,6.708802e+07,0.000000,17000.000000,19170.000000,10.000000,4000.000000


In [5]:
df.describe(include="object")

,EDUCATION,MARITAL_STATUS,REG_ADDRESS_PROVINCE,FACT_ADDRESS_PROVINCE,POSTAL_ADDRESS_PROVINCE,GEN_INDUSTRY,GEN_TITLE,JOB_DIR,FAMILY_INCOME
count,15223,15223,15223,15223,15223,13856,13856,13856,15223
unique,7,5,81,81,80,31,12,10,5
top,Среднее специальное,Состою в браке,Краснодарский край,Краснодарский край,Краснодарский край,Торговля,Специалист,Участие в основ. деятельности,от 10000 до 20000 руб.
freq,6518,9416,674,674,674,2385,7010,11452,7077


**Замечания**:
* WORK_TIME - Есть слишком большие значения WORK_TIME (тысячи месяцев работы) и WORK_TIME, где количество проработанных месяцев больше возраста человека или подозрительно близко к нему. Избавимся от таких данных ниже. Также есть пропуски - заполним их нулями.
* PERSONAL_INCOME - Есть несколько наблюдений, где зарплата меньше 4 000 руб. Будем считать такие наблюдения ошибочными и избавимся от них ниже.

In [6]:
# Удалим вероятно некорректные зарплаты (оставим зарплаты > 4 000 руб.)
df_clean = df.query("PERSONAL_INCOME > 4000 | PERSONAL_INCOME.isnull()", engine='python')

# Избавимся от неверного или ложного WORK_TIME 
# (Если допустим, что человек работает с 12 лет на одном и том же месте, а WORK_TIME все равно больше)
df_clean = df_clean.query("AGE * 12 >= WORK_TIME + 12 * 12 | WORK_TIME.isnull()", engine='python')

# Заполним нулями неизвестные WORK_TIME значения
df_clean["WORK_TIME"].fillna(0, inplace=True)
# Заполним 'Неизвестно' неизвестные  значения
df_clean["GEN_INDUSTRY"].fillna("Неизвестно", inplace=True)
df_clean["GEN_TITLE"].fillna("Неизвестно", inplace=True)
df_clean["JOB_DIR"].fillna("Неизвестно", inplace=True)

df_clean.replace(REMAP_DICT, inplace=True)

In [7]:
print(f"Initial size:\t{df.shape[0]} samples\nAfter cleaning:\t{df_clean.shape[0]} samples")

Initial size:	15223 samples
After cleaning:	15060 samples


In [8]:
df_clean

,AGE,GENDER,EDUCATION,MARITAL_STATUS,CHILD_TOTAL,DEPENDANTS,IS_WORKING,IS_PENSIONER,REG_ADDRESS_PROVINCE,FACT_ADDRESS_PROVINCE,POSTAL_ADDRESS_PROVINCE,HAS_FLAT,OWN_AUTO,ID_LOAN,ID_CLIENT,CLOSED_CREDIT,GEN_INDUSTRY,GEN_TITLE,JOB_DIR,WORK_TIME,AGREEMENT_RK,TARGET,FAMILY_INCOME,PERSONAL_INCOME,CREDIT,TERM,FIRST_PAYMENT
0,42,1,Среднее,Не состоял в браке,1,0,1,0,Московская область,Московская область,Московская область,1,0,1753791446,106805103,0,Другие сферы,Работник сферы услуг,Участие в основ. деятельности,3.0,60099204,1,от 20000 до 50000 руб.,25000.0,5588.0,6,1000.0
1,28,1,Среднее специальное,Состою в браке,1,1,1,0,Читинская область,Читинская область,Читинская область,0,0,1753796120,106809308,0,Торговля,Специалист,Участие в основ. деятельности,5.0,62244665,0,от 10000 до 20000 руб.,10000.0,19498.0,12,0.0
2,64,0,Среднее специальное,Состою в браке,2,0,1,1,Иркутская область,Иркутская область,Иркутская область,0,1,1753792244,106805867,1,Другие сферы,Руководитель высшего звена,Участие в основ. деятельности,360.0,61050759,0,от 20000 до 50000 руб.,30000.0,15470.0,3,15000.0
3,54,1,Среднее специальное,Состою в браке,0,0,1,0,Новосибирская область,Новосибирская область,Новосибирская область,1,1,1753795547,106808779,0,Государственная служба,Специалист,Участие в основ. деятельности,3.0,62079659,0,от 20000 до 50000 руб.,25000.0,13960.0,6,2500.0
4,26,0,Среднее специальное,Состою в браке,1,1,1,0,Красноярский край,Красноярский край,Красноярский край,1,0,1753802107,106814289,1,Другие сферы,Специалист,Участие в основ. деятельности,12.0,66583553,0,от 10000 до 20000 руб.,15000.0,11890.0,6,8000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21529,26,1,Среднее,Не состоял в браке,1,1,1,0,Чувашия,Чувашия,Чувашия,0,0,1753803070,106814680,1,Сборочные производства,Специалист,Участие в основ. деятельности,36.0,66751099,0,от 5000 до 10000 руб.,8000.0,11750.0,4,1900.0
21531,26,0,Среднее специальное,Состою в браке,0,0,1,0,Карелия,Карелия,Карелия,0,0,1753799545,106812464,0,Торговля,Специалист,Участие в основ. деятельности,24.0,64562377,0,от 20000 до 50000 руб.,12000.0,12350.0,6,1380.0
21532,30,1,Среднее специальное,Не состоял в браке,0,0,1,0,Белгородская область,Белгородская область,Белгородская область,0,0,1753796042,106809255,0,Торговля,Специалист,Участие в основ. деятельности,36.0,62236542,0,от 5000 до 10000 руб.,9000.0,4915.0,10,2000.0
21533,25,0,Среднее специальное,Состою в браке,0,0,1,0,Кабардино-Балкария,Кабардино-Балкария,Кабардино-Балкария,0,0,1753802864,106814593,1,Транспорт,Специалист,Участие в основ. деятельности,36.0,66739926,0,от 10000 до 20000 руб.,12000.0,5860.0,3,2000.0


In [9]:
# df_clean.to_csv("data/bank_dataset.csv", index=False)

## Задание 2

При помощи инструмента Streamlit проведите разведочный анализ данных. В него может входить:

* построение графиков распределений признаков
* построение матрицы корреляций
* построение графиков зависимостей целевой переменной и признаков
* вычисление числовых характеристик распределения числовых столбцов (среднее, min, max, медиана и так далее)
* любые другие ваши идеи приветствуются!

[Пример Streamlit-приложения](https://rateyourflight.streamlit.app) с разведочным анализом, прогнозом модели и оценкой ее результатов.

**Весь код к Streamlit приложению см. в app.py и utils.py**

## БОНУС: Обучение модели

In [10]:
X = df_clean.drop(columns=["REG_ADDRESS_PROVINCE", "FACT_ADDRESS_PROVINCE",
                          "AGREEMENT_RK", "ID_LOAN", "ID_CLIENT"]).copy()
X = X.replace(REMAP_DICT)
y = X["TARGET"]
X = X.drop(columns=["TARGET"])

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.9, stratify=y,
                                                    shuffle=True, random_state=42)

In [12]:
categorical_features_indices = np.where((X.dtypes != float) & (X.dtypes != int))[0]

model = CatBoostClassifier(
    iterations=500,
    learning_rate=0.1,
    max_depth=8,
    custom_loss=[metrics.F1()],
    random_seed=42,
    class_weights=(1, 8),
    logging_level='Silent',
)

In [13]:
model.fit(
    X_train, y_train,
    cat_features=categorical_features_indices,
    eval_set=(X_test, y_test),
    plot=False
)

In [14]:
accuracy_score(y_test, model.predict(X_test))

0.603585657370518

In [ ]:
# model.save_model("model_weights/model.weights")